In this notebook we will take the seasonal differentials of a teams stats and compare them against the opposing teams seasonal differentials

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from sklearn import neighbors
from sklearn.metrics import log_loss
#from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression

pd.set_option('display.max_columns', 500)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
sys.path

from src.data import make_dataset
currSeason = 2019
data = make_dataset.get_boxscore_dataset_v1(season=currSeason)
data.head()

,season,daynum,numot,tourney,team1,team2,score1,score2,loc,team1win,seed1,seednum1,seed2,seednum2,confabbrev1,conf_descr1,confabbrev2,conf_descr2,teamname1,firstd1season1,lastd1season1,teamname2,firstd1season2,lastd1season2,seeddiff,ID,score_team_mean1,score_team_std1,fgm_team_mean1,fgm_team_std1,fga_team_mean1,fga_team_std1,fgm3_team_mean1,fgm3_team_std1,fga3_team_mean1,fga3_team_std1,ftm_team_mean1,ftm_team_std1,fta_team_mean1,fta_team_std1,or_team_mean1,or_team_std1,dr_team_mean1,dr_team_std1,ast_team_mean1,ast_team_std1,to_team_mean1,to_team_std1,stl_team_mean1,stl_team_std1,blk_team_mean1,blk_team_std1,pf_team_mean1,pf_team_std1,score_opp_mean1,score_opp_std1,fgm_opp_mean1,fgm_opp_std1,fga_opp_mean1,fga_opp_std1,fgm3_opp_mean1,fgm3_opp_std1,fga3_opp_mean1,fga3_opp_std1,ftm_opp_mean1,ftm_opp_std1,fta_opp_mean1,fta_opp_std1,or_opp_mean1,or_opp_std1,dr_opp_mean1,dr_opp_std1,ast_opp_mean1,ast_opp_std1,to_opp_mean1,to_opp_std1,stl_opp_mean1,stl_opp_std1,blk_opp_mean1,blk_opp_std1,pf_opp_mean1,pf_opp_std1,score_team_mean2,score_team_std2,fgm_team_mean2,fgm_team_std2,fga_team_mean2,fga_team_std2,fgm3_team_mean2,fgm3_team_std2,fga3_team_mean2,fga3_team_std2,ftm_team_mean2,ftm_team_std2,fta_team_mean2,fta_team_std2,or_team_mean2,or_team_std2,dr_team_mean2,dr_team_std2,ast_team_mean2,ast_team_std2,to_team_mean2,to_team_std2,stl_team_mean2,stl_team_std2,blk_team_mean2,blk_team_std2,pf_team_mean2,pf_team_std2,score_opp_mean2,score_opp_std2,fgm_opp_mean2,fgm_opp_std2,fga_opp_mean2,fga_opp_std2,fgm3_opp_mean2,fgm3_opp_std2,fga3_opp_mean2,fga3_opp_std2,ftm_opp_mean2,ftm_opp_std2,fta_opp_mean2,fta_opp_std2,or_opp_mean2,or_opp_std2,dr_opp_mean2,dr_opp_std2,ast_opp_mean2,ast_opp_std2,to_opp_mean2,to_opp_std2,stl_opp_mean2,stl_opp_std2,blk_opp_mean2,blk_opp_std2,pf_opp_mean2,pf_opp_std2
0,2019,1,0,0,1104,1380,82,62,1104,1,NaN,NaN,NaN,NaN,sec,Southeastern Conference,swac,Southwest Athletic Conference,Alabama,1985,2019,Southern Univ,1985,2019,NaN,2019_1104_1380,71.787879,10.178892,25.060606,3.418145,56.848485,5.574054,7.060606,2.461445,20.848485,4.388345,14.606061,6.229664,21.878788,8.572330,11.151515,4.280196,26.424242,4.589596,12.151515,3.183171,13.606061,3.561420,4.606061,2.423058,4.666667,2.188988,16.969697,3.836379,71.303030,8.694412,26.060606,3.490517,59.606061,7.664608,6.424242,2.462599,19.424242,3.816333,12.757576,5.344567,18.000000,7.512490,9.969697,3.720592,24.454545,3.614994,11.333333,2.857738,11.878788,3.637973,6.333333,3.149074,4.303030,2.455668,19.030303,4.551681,64.612903,11.839700,23.354839,4.215435,53.483871,7.411122,5.548387,3.171106,17.032258,4.909066,12.354839,4.513302,18.483871,6.766934,10.000000,3.829708,20.612903,4.751683,11.354839,4.062416,15.870968,4.310777,6.580645,2.813944,2.548387,1.840991,22.645161,5.583000,74.032258,13.876800,24.741935,4.979744,51.129032,6.422574,6.677419,2.329336,18.193548,4.571064,17.870968,6.141346,25.838710,7.925052,10.096774,4.346300,23.161290,4.913226,12.548387,5.078311,15.129032,3.981138,6.354839,3.028183,3.354839,2.374144,17.741935,3.785655
1,2019,1,0,0,1113,1168,102,94,1113,1,X11a,11.0,NaN,NaN,pac_twelve,Pacific-12 Conference,big_west,Big West Conference,Arizona St,1985,2019,CS Fullerton,1985,2019,NaN,2019_1113_1168,77.812500,10.626921,26.593750,4.187996,59.468750,6.710533,7.250000,2.243270,21.250000,4.464917,17.375000,5.314314,25.906250,6.745892,12.156250,3.520533,27.812500,5.515009,13.593750,3.800334,13.593750,3.884289,6.218750,2.392655,3.218750,1.995711,19.906250,4.168696,73.062500,10.892251,24.218750,4.375403,58.687500,6.377569,8.562500,3.026096,25.500000,4.464917,16.062500,5.604937,22.656250,7.069286,10.000000,3.482676,25.218750,5.410470,13.281250,3.284709,13.906250,3.411171,6.343750,2.858032,3.468750,2.514274,21.656250,4.084663,70.129032,12.901529,24.838710,4.435439,55.741935,6.082586,5.290323,2.734880,17.064516,5.065804,15.161290,6.314516,21.483871,8.759265,8.451613,3.713388,26.967742,4.840688,11.225806,4.558579,13.612903,3.272893,5.870968,2.837157,3.290323,2.116499,19.161290,5.06

In [2]:
submissionData = make_dataset.get_boxscore_dataset_v1(season=currSeason, final_prediction=True)
submissionData.head()

,ID,season,team1,team2,score_team_mean1,score_team_std1,fgm_team_mean1,fgm_team_std1,fga_team_mean1,fga_team_std1,fgm3_team_mean1,fgm3_team_std1,fga3_team_mean1,fga3_team_std1,ftm_team_mean1,ftm_team_std1,fta_team_mean1,fta_team_std1,or_team_mean1,or_team_std1,dr_team_mean1,dr_team_std1,ast_team_mean1,ast_team_std1,to_team_mean1,to_team_std1,stl_team_mean1,stl_team_std1,blk_team_mean1,blk_team_std1,pf_team_mean1,pf_team_std1,score_opp_mean1,score_opp_std1,fgm_opp_mean1,fgm_opp_std1,fga_opp_mean1,fga_opp_std1,fgm3_opp_mean1,fgm3_opp_std1,fga3_opp_mean1,fga3_opp_std1,ftm_opp_mean1,ftm_opp_std1,fta_opp_mean1,fta_opp_std1,or_opp_mean1,or_opp_std1,dr_opp_mean1,dr_opp_std1,ast_opp_mean1,ast_opp_std1,to_opp_mean1,to_opp_std1,stl_opp_mean1,stl_opp_std1,blk_opp_mean1,blk_opp_std1,pf_opp_mean1,pf_opp_std1,score_team_mean2,score_team_std2,fgm_team_mean2,fgm_team_std2,fga_team_mean2,fga_team_std2,fgm3_team_mean2,fgm3_team_std2,fga3_team_mean2,fga3_team_std2,ftm_team_mean2,ftm_team_std2,fta_team_mean2,fta_team_std2,or_team_mean2,or_team_std2,dr_team_mean2,dr_team_std2,ast_team_mean2,ast_team_std2,to_team_mean2,to_team_std2,stl_team_mean2,stl_team_std2,blk_team_mean2,blk_team_std2,pf_team_mean2,pf_team_std2,score_opp_mean2,score_opp_std2,fgm_opp_mean2,fgm_opp_std2,fga_opp_mean2,fga_opp_std2,fgm3_opp_mean2,fgm3_opp_std2,fga3_opp_mean2,fga3_opp_std2,ftm_opp_mean2,ftm_opp_std2,fta_opp_mean2,fta_opp_std2,or_opp_mean2,or_opp_std2,dr_opp_mean2,dr_opp_std2,ast_opp_mean2,ast_opp_std2,to_opp_mean2,to_opp_std2,stl_opp_mean2,stl_opp_std2,blk_opp_mean2,blk_opp_std2,pf_opp_mean2,pf_opp_std2
0,2019_1101_1113,2019,1101,1113,71.724138,8.871691,25.344828,3.78225,55.241379,3.888216,7.241379,2.444457,18.896552,4.498768,13.793103,5.246627,19.068966,6.808246,9.068966,2.840159,22.896552,4.434804,14.62069,3.374418,11.655172,4.600278,8.0,3.011881,2.551724,1.723497,19.137931,4.801221,64.896552,10.223729,22.482759,3.841695,51.137931,5.514424,5.724138,2.447981,17.103448,4.295719,14.206897,7.403567,19.655172,8.221383,8.827586,3.586373,23.758621,3.439842,11.103448,3.894546,15.724138,4.34163,5.551724,2.693726,2.965517,1.375364,18.068966,3.909065,77.812500,10.626921,26.593750,4.187996,59.468750,6.710533,7.250000,2.243270,21.250000,4.464917,17.375000,5.314314,25.906250,6.745892,12.156250,3.520533,27.812500,5.515009,13.593750,3.800334,13.593750,3.884289,6.218750,2.392655,3.218750,1.995711,19.906250,4.168696,73.062500,10.892251,24.218750,4.375403,58.687500,6.377569,8.562500,3.026096,25.500000,4.464917,16.062500,5.604937,22.656250,7.069286,10.000000,3.482676,25.218750,5.410470,13.281250,3.284709,13.906250,3.411171,6.343750,2.858032,3.468750,2.514274,21.656250,4.084663
1,2019_1101_1120,2019,1101,1120,71.724138,8.871691,25.344828,3.78225,55.241379,3.888216,7.241379,2.444457,18.896552,4.498768,13.793103,5.246627,19.068966,6.808246,9.068966,2.840159,22.896552,4.434804,14.62069,3.374418,11.655172,4.600278,8.0,3.011881,2.551724,1.723497,19.137931,4.801221,64.896552,10.223729,22.482759,3.841695,51.137931,5.514424,5.724138,2.447981,17.103448,4.295719,14.206897,7.403567,19.655172,8.221383,8.827586,3.586373,23.758621,3.439842,11.103448,3.894546,15.724138,4.34163,5.551724,2.693726,2.965517,1.375364,18.068966,3.909065,78.882353,11.493779,27.029412,5.131515,60.441176,8.749688,11.411765,3.429972,29.970588,5.718048,13.411765,5.565683,18.735294,6.345167,11.735294,4.724590,21.941176,4.492021,14.411765,4.546845,12.147059,4.452662,9.294118,3.572024,4.764706,2.362059,18.382353,3.701062,68.588235,12.274926,23.235294,4.499554,52.941176,6.678510,8.323529,2.671064,23.382353,4.923750,13.794118,5.596424,19.352941,7.865854,10.764706,3.977656,24.176471,4.003563,12.029412,3.214412,17.588235,4.812343,5.911765,2.821644,3.676471,2.446031,17.205882,3.747964
2,2019_1101_1124,2019,1101,1124,71.724138,8.871691,25.344828,3.78225,55.241379,3.888216,7.241379,2.444457,18.896552,4.498768,13.793103,5.246627,19.068966,6.808246,9.068966,2.840159,22.896552,4.434804,14.62069,3.374418,11.655172,4.600278,8.0,3.011881,2.55

In [3]:
def addDifferentialsToDataframe(dataframe):
    cols = dataframe.columns
    diffCols = [s for s in cols if "mean" in s]
    diffCats=[]
    for col in diffCols:
        cat = col.split('_')[0]
        if cat not in diffCats:
            diffCats.append(cat)
    
    
    diffDict = {}
    for team in [1, 2]:
        for cat in diffCats:
            diffDict[cat + '_diff_' + str(team)] = dataframe[cat + '_team_mean' + str(team)] - dataframe[cat + '_opp_mean' + str(team)]
            dataframe.drop(columns =[cat + '_team_mean' + str(team),
                                cat + '_team_std' + str(team), 
                                cat + '_opp_mean' + str(team),
                                cat + '_opp_std' + str(team)],
                      inplace = True)
    dfDiff = pd.DataFrame(diffDict, columns = diffDict.keys())
    dataframe = pd.concat([dataframe, dfDiff], axis=1)
    return dataframe

data = addDifferentialsToDataframe(data)
data.drop(columns =['season',
                    'daynum',
                    'numot',
                    'score1',
                    'score2',
                    'loc',
                    'seed1',
                    'seednum1',
                    'seed2',
                    'seednum2',
                    'confabbrev1',
                    'conf_descr1',
                    'confabbrev2',
                    'conf_descr2',
                    'teamname1',
                    'firstd1season1',
                    'lastd1season1',
                    'teamname2',
                    'firstd1season2',
                    'lastd1season2',
                    'seeddiff'],
                  inplace = True)
data.head()

,tourney,team1,team2,team1win,ID,score_diff_1,fgm_diff_1,fga_diff_1,fgm3_diff_1,fga3_diff_1,ftm_diff_1,fta_diff_1,or_diff_1,dr_diff_1,ast_diff_1,to_diff_1,stl_diff_1,blk_diff_1,pf_diff_1,score_diff_2,fgm_diff_2,fga_diff_2,fgm3_diff_2,fga3_diff_2,ftm_diff_2,fta_diff_2,or_diff_2,dr_diff_2,ast_diff_2,to_diff_2,stl_diff_2,blk_diff_2,pf_diff_2
0,0,1104,1380,1,2019_1104_1380,0.484848,-1.000000,-2.757576,0.636364,1.424242,1.848485,3.878788,1.181818,1.969697,0.818182,1.727273,-1.727273,0.363636,-2.060606,-9.419355,-1.387097,2.354839,-1.129032,-1.161290,-5.516129,-7.354839,-0.096774,-2.548387,-1.193548,0.741935,0.225806,-0.806452,4.903226
1,0,1113,1168,1,2019_1113_1168,4.750000,2.375000,0.781250,-1.312500,-4.250000,1.312500,3.250000,2.156250,2.593750,0.312500,-0.312500,-0.125000,-0.250000,-1.750000,-1.967742,-0.935484,-3.354839,-0.903226,-2.193548,0.806452,0.354839,-1.258065,1.032258,-1.129032,1.645161,0.032258,0.516129,0.064516
2,0,1119,1265,1,2019_1119_1265,-2.838710,-0.870968,2.709677,0.096774,3.000000,-1.193548,-1.709677,0.129032,-2.451613,1.354839,-1.612903,-0.741935,-2.741935,1.290323,-1.967742,0.354839,0.322581,-1.064516,-3.451613,-1.612903,-1.870968,-0.709677,-0.838710,-1.032258,-0.225806,-0.096774,-1.483871,1.354839
3,0,1120,1375,1,2019_1120_1375,10.294118,3.794118,7.500000,3.088235,6.588235,-0.382353,-0.617647,0.970588,-2.235294,2.382353,-5.441176,3.382353,1.088235,1.176471,-1.750000,-0.781250,-1.593750,0.812500,1.406250,-1.000000,-1.375000,-2.468750,-1.375000,-0.562500,-0.125000,-0.062500,-0.812500,1.312500
4,0,1123,1232,1,2019_1123_1232,2.187500,0.656250,-5.062500,-0.875000,-2.781250,1.750000,2.625000,-1.937500,3.375000,0.843750,1.656250,-0.187500,0.125000,-1.625000,-3.344828,-0.862069,0.413793,-1.103448,-4.172414,-0.517241,-1.068966,-0.896552,-2.068966,-2.275862,-0.551724,0.758621,-1.137931,0.724138


In [4]:
submissionData = addDifferentialsToDataframe(submissionData)
submissionData.drop(columns = ['season'], inplace=True)
submissionData.head()

,ID,team1,team2,score_diff_1,fgm_diff_1,fga_diff_1,fgm3_diff_1,fga3_diff_1,ftm_diff_1,fta_diff_1,or_diff_1,dr_diff_1,ast_diff_1,to_diff_1,stl_diff_1,blk_diff_1,pf_diff_1,score_diff_2,fgm_diff_2,fga_diff_2,fgm3_diff_2,fga3_diff_2,ftm_diff_2,fta_diff_2,or_diff_2,dr_diff_2,ast_diff_2,to_diff_2,stl_diff_2,blk_diff_2,pf_diff_2
0,2019_1101_1113,1101,1113,6.827586,2.862069,4.103448,1.517241,1.793103,-0.413793,-0.586207,0.241379,-0.862069,3.517241,-4.068966,2.448276,-0.413793,1.068966,4.750000,2.375000,0.781250,-1.312500,-4.250000,1.312500,3.250000,2.156250,2.593750,0.312500,-0.312500,-0.125000,-0.250000,-1.750000
1,2019_1101_1120,1101,1120,6.827586,2.862069,4.103448,1.517241,1.793103,-0.413793,-0.586207,0.241379,-0.862069,3.517241,-4.068966,2.448276,-0.413793,1.068966,10.294118,3.794118,7.500000,3.088235,6.588235,-0.382353,-0.617647,0.970588,-2.235294,2.382353,-5.441176,3.382353,1.088235,1.176471
2,2019_1101_1124,1101,1124,6.827586,2.862069,4.103448,1.517241,1.793103,-0.413793,-0.586207,0.241379,-0.862069,3.517241,-4.068966,2.448276,-0.413793,1.068966,4.468750,2.281250,2.875000,0.843750,2.593750,-0.937500,-1.218750,3.406250,2.906250,1.000000,0.875000,0.156250,0.531250,0.750000
3,2019_1101_1125,1101,1125,6.827586,2.862069,4.103448,1.517241,1.793103,-0.413793,-0.586207,0.241379,-0.862069,3.517241,-4.068966,2.448276,-0.413793,1.068966,12.200000,4.333333,-0.033333,2.700000,5.833333,0.833333,0.233333,-0.600000,3.733333,7.433333,-0.633333,0.466667,0.100000,-0.833333
4,2019_1101_1133,1101,1133,6.827586,2.862069,4.103448,1.517241,1.793103,-0.413793,-0.586207,0.241379,-0.862069,3.517241,-4.068966,2.448276,-0.413793,1.068966,1.181818,0.818182,-0.757576,-0.151515,-2.636364,-0.303030,0.303030,-0.121212,0.848485,0.151515,0.242424,-0.454545,1.272727,-1.121212


Now we are going to duplicate all of this training data, but reverse team1 and team2. The team1win column will need to be reversed as well (i.e. if the current value is 1 the new value is 0 and vice versa)

In [5]:
def inverse1and0(v):
    if (v == 1):
        return 0
    else:
        return 1

duplicateData = data.copy()
duplicateData['team1win']=duplicateData['team1win'].map(inverse1and0)
columns = list(duplicateData)

swapped = []
for col in columns:
    if col in swapped:
        continue
    if col.endswith('1'):
        col2 = col[:-1] + '2'
        col1idx = columns.index(col)
        col2idx = columns.index(col2)
        columns[col1idx], columns[col2idx] = columns[col2idx], columns[col1idx]
        swapped.append(col)

duplicateData.columns = columns
duplicateData.head()


,tourney,team2,team1,team1win,ID,score_diff_2,fgm_diff_2,fga_diff_2,fgm3_diff_2,fga3_diff_2,ftm_diff_2,fta_diff_2,or_diff_2,dr_diff_2,ast_diff_2,to_diff_2,stl_diff_2,blk_diff_2,pf_diff_2,score_diff_1,fgm_diff_1,fga_diff_1,fgm3_diff_1,fga3_diff_1,ftm_diff_1,fta_diff_1,or_diff_1,dr_diff_1,ast_diff_1,to_diff_1,stl_diff_1,blk_diff_1,pf_diff_1
0,0,1104,1380,0,2019_1104_1380,0.484848,-1.000000,-2.757576,0.636364,1.424242,1.848485,3.878788,1.181818,1.969697,0.818182,1.727273,-1.727273,0.363636,-2.060606,-9.419355,-1.387097,2.354839,-1.129032,-1.161290,-5.516129,-7.354839,-0.096774,-2.548387,-1.193548,0.741935,0.225806,-0.806452,4.903226
1,0,1113,1168,0,2019_1113_1168,4.750000,2.375000,0.781250,-1.312500,-4.250000,1.312500,3.250000,2.156250,2.593750,0.312500,-0.312500,-0.125000,-0.250000,-1.750000,-1.967742,-0.935484,-3.354839,-0.903226,-2.193548,0.806452,0.354839,-1.258065,1.032258,-1.129032,1.645161,0.032258,0.516129,0.064516
2,0,1119,1265,0,2019_1119_1265,-2.838710,-0.870968,2.709677,0.096774,3.000000,-1.193548,-1.709677,0.129032,-2.451613,1.354839,-1.612903,-0.741935,-2.741935,1.290323,-1.967742,0.354839,0.322581,-1.064516,-3.451613,-1.612903,-1.870968,-0.709677,-0.838710,-1.032258,-0.225806,-0.096774,-1.483871,1.354839
3,0,1120,1375,0,2019_1120_1375,10.294118,3.794118,7.500000,3.088235,6.588235,-0.382353,-0.617647,0.970588,-2.235294,2.382353,-5.441176,3.382353,1.088235,1.176471,-1.750000,-0.781250,-1.593750,0.812500,1.406250,-1.000000,-1.375000,-2.468750,-1.375000,-0.562500,-0.125000,-0.062500,-0.812500,1.312500
4,0,1123,1232,0,2019_1123_1232,2.187500,0.656250,-5.062500,-0.875000,-2.781250,1.750000,2.625000,-1.937500,3.375000,0.843750,1.656250,-0.187500,0.125000,-1.625000,-3.344828,-0.862069,0.413793,-1.103448,-4.172414,-0.517241,-1.068966,-0.896552,-2.068966,-2.275862,-0.551724,0.758621,-1.137931,0.724138


In [6]:
tourneyData = data.loc[data['tourney'] == 1]

concat_data = pd.concat([data, duplicateData], sort=False)
regSeasonData = concat_data.loc[concat_data['tourney'] == 0]

regSeasonData.drop(columns = ['tourney'], inplace = True)
tourneyData.drop(columns = ['tourney'], inplace = True)

x_train = np.array(regSeasonData.drop(columns = ['team1win', 'team1', 'team2', 'ID']))
y_train = np.array(regSeasonData['team1win'])

y_test = np.array(tourneyData['team1win'])

#x_test = np.array(tourneyData.drop(columns = ['team1win', 'team1', 'team2', 'ID']))
x_test = np.array(submissionData.drop(columns = ['team1', 'team2', 'ID']))


/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [7]:
#for k in range(1,10):
#    k = k*10
clf = neighbors.KNeighborsClassifier(n_neighbors=20, weights='distance')
clf.fit(x_train, y_train)
y_pred = clf.predict_proba(x_test)[:, 0]
#    test_loss = log_loss(y_test,y_pred)
#    accuracy = clf.score(x_test, y_test)
#    print(' logloss = ', test_loss, ' accuracy = ', accuracy)

#clf = LogisticRegression(solver='liblinear').fit(x_train, y_train)
#y_pred = clf.predict_proba(x_test)[:, 0]
#print(y_pred)
#print(y_test)
#test_loss = log_loss(y_test, y_pred)
#accuracy = clf.score(x_test, y_test)
#print(' logloss = ', test_loss, ' accuracy = ', accuracy)

#ID = data.loc[data['tourney'] == 1, 'ID'].values
ID = submissionData['ID'].values

print(len(ID), len(y_pred))
df_pred = pd.DataFrame({'ID':ID, 'Pred':y_pred})
df_pred.head()
    
#save prediction csv
pred_dir = module_path + '/models/knn20/' + str(currSeason) + '/'
if not os.path.exists(pred_dir):
    print('making dir')
    os.makedirs(pred_dir)
    
df_pred[['ID', 'Pred']].to_csv(pred_dir + 'pred.csv', index=False)


2278 2278
making dir
